In [1]:
#mount
import os
cwd = os.getcwd()
if cwd[0:2] != "C:":
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
import sys
root_dir="/content/gdrive/My Drive/Projects/flower"
sys.path.append(root_dir)

%cd "/content/gdrive/My Drive/Projects/flower/wm"

/content/gdrive/My Drive/Projects/flower/wm


In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [72]:
import pandas as pd
import numpy as np
import datetime as dt
import importlib
import tradestats
import pandas as pd
from datetime import datetime
import sys

In [5]:
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz

In [6]:
importlib.reload(tradestats)
from tradestats import *

In [7]:
importlib.reload(tradestats)
from tradestats import *
from tradestats import rose
prices = loaddata_1W('mbk_w.csv')

In [127]:
importlib.reload(tradestats)
from tradestats import *
from tradestats import rose

history = 10
ignore_max = 11
entry_max = ignore_max
filenames = ['mbk_w.csv','mbk_d.csv','alr_w.csv','alr_d.csv','pko_w.csv','pko_d.csv','peo_w.csv','peo_d.csv','pkn_w.csv','pkn_d.csv']
progress = 0
progress_max = ((ignore_max)/2)*(ignore_max-1)*len(filenames)
stats = pd.DataFrame()

for filename in filenames:
  print('')
  print(filename)
  if '_d.'in filename:
    prices = loaddata_1D(filename)
  elif '_w.'in filename:
    prices = loaddata_1W(filename)
  else:
    print('file timeframe not supported')
    continue
  for ignore in range(1,ignore_max):
    for entry in range(ignore,entry_max):
      periods = [2]
      dictList = []
      colFeat = []
      keyList = []
      roseDict   = rose(prices, periods,history,ignore,entry)
      dictList.append(roseDict.df); colFeat.append('rose'); keyList.append(periods)
      masterFrame = pd.DataFrame(index=prices.index)
      masterFrame[['year','date','close']] = prices[['year','date','close']]

      for i in range(0,len(dictList)):#iteracja po funkcjach
        for j in keyList[i]: #iteracja po okresach
          for k in list(dictList[i][j]): #iteracja po kolumnach danej funkcji w okresie
              colID = colFeat[i] + str(j) + k
              masterFrame[colID] = dictList[i][j][k]
      masterFrame = masterFrame.dropna()
      sum = masterFrame['rose2profit'].agg('sum')
      sum1990 = masterFrame.loc[(masterFrame.year>=1990) & (masterFrame.year<2000),'rose2profit'].agg('sum')
      sum2000 = masterFrame.loc[(masterFrame.year>=2000) & (masterFrame.year<2010),'rose2profit'].agg('sum')
      sum2010 = masterFrame.loc[(masterFrame.year>=2010) & (masterFrame.year<2020),'rose2profit'].agg('sum')
      sum2020 = masterFrame.loc[(masterFrame.year>=2020) & (masterFrame.year<2030),'rose2profit'].agg('sum')
      sumXX = masterFrame.loc[(masterFrame.year>=2000) & (masterFrame.year<2030),'rose2profit'].agg('sum')

      xx = masterFrame.groupby('year').rose2profit.agg('sum')
      df = xx.to_frame().T
      df.insert(loc=0, column='sum2020', value=[sum2020])
      df.insert(loc=0, column='sum2010', value=[sum2010])
      df.insert(loc=0, column='sum2000', value=[sum2000])
      df.insert(loc=0, column='sum1990', value=[sum1990])
      df.insert(loc=0, column='sumXX', value=[sumXX])
      df.insert(loc=0, column='sum', value=[sum])
      df.insert(loc=0, column='entry', value=[entry])
      df.insert(loc=0, column='ignore', value=[ignore])
      df.insert(loc=0, column='file', value=[filename])
      #df['ignore'] = ignore
      #df['entry'] = entry
      #df['file'] = filename
      df.reset_index(inplace = True, drop = True)
      if len(stats.index) == 0:
        stats = df
      else:
        stats = pd.concat([stats, df], ignore_index=True)
      progress +=1
      sys.stdout.write("\r%d%%" % round(progress/progress_max*100,0))
      sys.stdout.flush()




mbk_w.csv
10%
mbk_d.csv
20%
alr_w.csv
30%
alr_d.csv
40%
pko_w.csv
50%
pko_d.csv
60%
peo_w.csv
70%
peo_d.csv
80%
pkn_w.csv
90%
pkn_d.csv
100%

In [128]:
stats

year,file,ignore,entry,sum,sumXX,sum1990,sum2000,sum2010,sum2020,1992,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,mbk_w.csv,1,1,5.410989,2.418708,2.992282,1.012185,-0.243893,1.650416,-0.052353,...,-0.525610,-0.165486,0.167939,-0.184762,-0.156725,-0.446888,1.048009,0.335874,0.635284,0.078137
1,mbk_w.csv,1,2,6.918212,3.579072,3.339140,1.507848,0.127751,1.943474,-0.052353,...,-0.488143,0.019129,0.202727,-0.189529,-0.165934,-0.167767,1.238101,0.304890,0.591139,-0.022890
2,mbk_w.csv,1,3,6.718245,3.558149,3.160096,1.975794,-0.006838,1.589193,-0.071827,...,-0.530919,-0.075115,0.122581,-0.047372,-0.162992,-0.267928,1.145659,0.210194,0.542143,-0.040876
3,mbk_w.csv,1,4,8.049664,4.528763,3.520901,2.197901,0.360725,1.970137,-0.128507,...,-0.384247,-0.096037,0.119539,0.007738,-0.030114,-0.175633,1.519824,0.058011,0.567935,0.000000
4,mbk_w.csv,1,5,7.988893,3.625695,4.363198,1.421387,0.386044,1.818264,-0.128507,...,-0.073680,-0.102158,0.192107,-0.000859,0.058001,-0.452039,1.537897,0.041715,0.690690,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,pkn_d.csv,8,9,2.642771,2.642771,0.000000,1.767391,1.137436,-0.262055,NaN,...,0.000000,0.000000,0.892461,0.000000,0.000000,-0.195686,0.002239,-0.068609,0.000000,0.000000
546,pkn_d.csv,8,10,2.280638,2.280638,0.000000,1.850777,-0.022512,0.452373,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.251169,0.000000,0.201205,0.000000,0.000000
547,pkn_d.csv,9,9,2.969979,2.969979,0.000000,2.026405,1.093218,-0.149644,NaN,...,0.000000,0.000000,0.892461,0.000000,0.000000,-0.195686,0.002239,0.043803,0.000000,0.000000
548,pkn_d.csv,9,10,2.302150,2.302150,0.000000,1.457466,0.392311,0.452373,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.251169,0.000000,0.201205,0.000000,0.000000


In [129]:
stats.to_csv(sep=';',path_or_buf='../Data/stats.csv',date_format="%Y-%m-%d",index = False,na_rep='')

In [100]:
type(sum)

numpy.float64

In [9]:
# prices.dropna(inplace=True)
masterFrame = pd.DataFrame(index=prices.index)
#masterFrame[['id','year','month','day','date','weekday','open','high','low','close','volume']] = prices[['id','year','month','day','date','weekday','open','high','low','close','volume']]
masterFrame[['year','date','close']] = prices[['year','date','close']]

In [10]:
for i in range(0,len(dictList)):#iteracja po funkcjach
    for j in keyList[i]: #iteracja po okresach
        for k in list(dictList[i][j]): #iteracja po kolumnach danej funkcji w okresie
            colID = colFeat[i] + str(j) + k
            masterFrame[colID] = dictList[i][j][k]

In [12]:
masterFrame = masterFrame.dropna()

In [13]:
masterFrame#.tail(100)

,year,date,close,rose2rose,rose2uptick_date,rose2uptick_close,rose2uptick_diff,rose2downtick_date,rose2downtick_close,rose2downtick_diff,rose2entry,rose2open_trade_date,rose2open_trade_close,rose2profit
0,1992,1992-10-05,3.919933,1,1900-01-01,0.000000,0.000000,1900-01-01,0.000000,0.000000,0,1900-01-01,0.0,0.0
1,1992,1992-10-12,3.600065,-2,1900-01-01,0.000000,0.000000,1900-01-01,0.000000,0.000000,0,1900-01-01,0.0,0.0
2,1992,1992-10-19,4.339995,2,1900-01-01,0.000000,0.000000,1992-10-12,3.600065,0.205532,1,1900-01-01,0.0,0.0
3,1992,1992-10-26,4.220030,-2,1992-10-19,4.339995,-0.027642,1992-10-12,3.600065,0.172209,0,1900-01-01,0.0,0.0
4,1992,1992-11-02,4.200015,-2,1992-10-19,4.339995,-0.032254,1992-10-12,3.600065,0.166650,0,1900-01-01,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1630,2024,2024-01-01,525.600000,-2,2023-12-18,548.000000,-0.040876,1900-01-01,0.000000,0.000000,0,1900-01-01,0.0,0.0
1631,2024,2024-01-08,526.600000,0,2023-12-18,548.000000,-0.039051,2024-01-01,525.600000,0.001903,0,1900-01-01,0.0,0.0
1632,2024,2024-01-15,499.300000,-2,2023-12-18,548.000000,-0.088869,1900-01-01,0.000000,0.000000,-1,1900-01-01,0.0,0.0
1633,2024,2024-01-22,506.800000,0,2023-12-18,548.000000,-0.075182,2024-01-15,499.300000,0.015021,0,1900-01-01,0.0,0.0


In [18]:
xx = masterFrame.groupby('year').rose2profit.agg('sum')

In [26]:
xx

year
1992   -0.133626
1993    2.405860
1994    0.897425
1995    0.252366
1996    0.072739
1997   -0.247984
1998    0.776744
1999    0.204119
2000   -0.098078
2001   -0.248301
2002   -0.153606
2003    0.310914
2004   -0.016397
2005    0.163426
2006    0.092362
2007    0.965556
2008   -0.398703
2009    0.970405
2010   -0.066743
2011   -0.153570
2012    0.069098
2013    0.419331
2014   -0.306175
2015   -0.177605
2016   -0.097134
2017    0.130127
2018   -0.138907
2019    0.002819
2020   -0.333350
2021    1.406444
2022   -0.010812
2023    0.581544
2024    0.000000
Name: rose2profit, dtype: float64

In [24]:
df = xx.to_frame().T

In [45]:
df

year,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
rose2profit,-0.133626,2.40586,0.897425,0.252366,0.072739,-0.247984,0.776744,0.204119,-0.098078,-0.248301,...,-0.177605,-0.097134,0.130127,-0.138907,0.002819,-0.33335,1.406444,-0.010812,0.581544,0.0


In [60]:
df.reset_index(inplace = True, drop = True)

In [110]:
df

year,file,ignore,entry,sum,1992,1993,1994,1995,1996,1997,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,mbk_w.csv,10,10,0.228832,0.0,1.892872,0.934047,-0.192952,0.944369,-0.247984,...,-0.384633,-0.073952,0.0,-0.00121,-0.088292,-0.298343,0.514436,-0.366781,0.273406,0.106114


In [62]:
df2 = df.copy()

In [63]:
df_row_merged = pd.concat([df, df2], ignore_index=True)

In [64]:
df_row_merged

year,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2018,2019,2020,2021,2022,2023,2024,ignore,entry,file
0,-0.133626,2.40586,0.897425,0.252366,0.072739,-0.247984,0.776744,0.204119,-0.098078,-0.248301,...,-0.138907,0.002819,-0.33335,1.406444,-0.010812,0.581544,0.0,1,1,mbk_w.csv
1,-0.133626,2.40586,0.897425,0.252366,0.072739,-0.247984,0.776744,0.204119,-0.098078,-0.248301,...,-0.138907,0.002819,-0.33335,1.406444,-0.010812,0.581544,0.0,1,1,mbk_w.csv


In [113]:
masterFrame.loc[masterFrame.year>=2022,'rose2profit'].agg('sum')

0.01273927468477426

In [106]:
masterFrame['rose2profit'].agg('sum')

5.743466877721712

In [48]:
masterFrame.to_csv(sep=';',path_or_buf='../Data/stats.csv',date_format="%Y-%m-%d",index = False,na_rep='')

rose2profit   -0.133626
Name: 1992, dtype: float64

In [ ]:
importlib.reload(tradestats)
from tradestats import *
starttime = datetime.now()
# alltrades = preparetrades_brut_tsl(masterFrame,[1],[0.001,0.002,0.003,0.004,0.005,0.006],[0.001,0.002,0.003,0.004,0.005,0.006],[0.001,0.002,0.003,0.004,0.005,0.006],2016,2017)
alltrades = preparetrades_brut_tp(masterFrame,[1],[0.001,0.002,0.003,0.004,0.005,0.006],[0.001,0.002,0.003,0.004,0.005,0.006],2016,2017)
endtime = datetime.now()
print(str(endtime - starttime))

In [ ]:
importlib.reload(tradestats)
from tradestats import *
trades = cleartrades_brut(alltrades.copy(),True)
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5],[13],[0.002],True)

In [ ]:
#____________________________________________________________________________________________________________________
#
#
alltrades.head(2)

In [ ]:
importlib.reload(tradestats)
from tradestats import *
stats1 = runstats_brut_v3(alltrades)